<a href="https://colab.research.google.com/github/NastaCom/Recommend_system/blob/main/course_work_v_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [181]:
!pip install implicit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [182]:
!pip install lightfm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [183]:
import numpy as np 
import pandas as pd

In [184]:
data = pd.read_csv('/content/retail_train.csv')
item_features = pd.read_csv('/content/product.csv')
user_features = pd.read_csv('/content/hh_demographic.csv')
test_df = pd.read_csv('/content/retail_test.csv')

# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)


# Важна схема обучения и валидации!
# -- давние покупки -- | -- 6 недель -- | -- 3 недель -- 
# подобрать размер 2-ого датасета (6 недель) --> learning curve (зависимость метрики recall@k от размера датасета)
val_lvl_1_size_weeks = 6
val_lvl_2_size_weeks = 3

data_train_lvl_1 = data[data['week_no'] < data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)]
data_val_lvl_1 = data[(data['week_no'] >= data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)) &
                      (data['week_no'] < data['week_no'].max() - (val_lvl_2_size_weeks))]

data_train_lvl_2 = data_val_lvl_1.copy()  # Для наглядности. Далее мы добавим изменения, и они будут отличаться
data_val_lvl_2 = data[data['week_no'] >= data['week_no'].max() - val_lvl_2_size_weeks]

data_train_lvl_1.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [185]:
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight
from implicit.bpr import BayesianPersonalizedRanking
from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
from lightfm import LightFM
from lightfm.evaluation import precision_at_k, recall_at_k

# Модель второго уровня
from lightgbm import LGBMClassifier

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

In [186]:
class MainRecommender:

    """Рекоммендации, которые можно получить из ALS

    Input
    -----
    user_item_matrix: pd.DataFrame
        Матрица взаимодействий user-item
    weighting: string
        Тип взвешивания, один из вариантов None, "bm25", "tfidf"
    fake_id: int
        идентификатор, которым заменялись редкие объекты, можно передать None?если такого объекта нет
    """

    def __init__(self, data, weighting=None, fake_id = 999999):

        # Топ покупок каждого юзера
        self.top_purchases = data.groupby(['user_id', 'item_id'])['quantity'].count().reset_index()
        self.top_purchases.sort_values('quantity', ascending=False, inplace=True)
        if fake_id is not None:
            self.top_purchases = self.top_purchases[self.top_purchases['item_id'] != fake_id]

        # Топ покупок по всему датасету
        self.overall_top_purchases = data.groupby('item_id')['quantity'].count().reset_index()
        self.overall_top_purchases.sort_values('quantity', ascending=False, inplace=True)
        if fake_id is not None:
            self.overall_top_purchases = self.overall_top_purchases[self.overall_top_purchases['item_id'] != fake_id]
        self.overall_top_purchases = self.overall_top_purchases.item_id.tolist()
        
        self.fake_id = fake_id
        self.user_item_matrix = self._prepare_matrix(data)  # pd.DataFrame
        self.id_to_itemid, self.id_to_userid, \
            self.itemid_to_id, self.userid_to_id = self._prepare_dicts(self.user_item_matrix)
        self.user_item_matrix = csr_matrix(self.user_item_matrix).tocsr()
        self.user_item_matrix_for_pred = self.user_item_matrix
        #Можно добавить дополнительное взвешивание
        if weighting == "bm25":
            self.user_item_matrix = bm25_weight(self.user_item_matrix.T).T.tocsr()   
        elif weighting == "tfidf":
            self.user_item_matrix = tfidf_weight(self.user_item_matrix.T).T.tocsr()

        self.model = self.fit(self.user_item_matrix)
        self.own_recommender = self.fit_own_recommender(self.user_item_matrix)

    @staticmethod
    def _prepare_matrix(data):        
        """Готовит user-item матрицу"""
#         popularity = data.groupby('item_id')['quantity'].sum().reset_index()
#         popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

#         top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()
        # Заведем фиктивный item_id (если юзер покупал товары не из топ-5000, то он "купил" такой товар)
#         data.loc[~data['item_id'].isin(top_5000), 'item_id'] = 999999
        
        user_item_matrix = pd.pivot_table(data,
                                          index='user_id', columns='item_id',
                                          values='quantity',  # Можно пробовать другие варианты
                                          aggfunc='count',
                                          fill_value=0
                                          )

        user_item_matrix = user_item_matrix.astype(float)  # необходимый тип матрицы для implicit

        
        return user_item_matrix

    @staticmethod
    def _prepare_dicts(user_item_matrix):
        """Подготавливает вспомогательные словари"""

        userids = user_item_matrix.index.values
        itemids = user_item_matrix.columns.values

        matrix_userids = np.arange(len(userids))
        matrix_itemids = np.arange(len(itemids))

        id_to_itemid = dict(zip(matrix_itemids, itemids))
        id_to_userid = dict(zip(matrix_userids, userids))

        itemid_to_id = dict(zip(itemids, matrix_itemids))
        userid_to_id = dict(zip(userids, matrix_userids))

        return id_to_itemid, id_to_userid, itemid_to_id, userid_to_id

    @staticmethod
    def fit_own_recommender(user_item_matrix):
        """Обучает модель, которая рекомендует товары, среди товаров, купленных юзером"""

        own_recommender = ItemItemRecommender(K=1, num_threads=4)
        own_recommender.fit(user_item_matrix)

        return own_recommender

    @staticmethod
    def fit(user_item_matrix, n_factors=20, regularization=0.01, iterations=15, num_threads=4):
        """Обучает ALS"""

        model = AlternatingLeastSquares(factors=n_factors,
                                        regularization=regularization,
                                        iterations=iterations,
                                        num_threads=num_threads)
        model.fit(user_item_matrix)

        return model

    def _update_dict(self, user_id):
        """Если появился новыю user / item, то нужно обновить словари"""

        if user_id not in self.userid_to_id.keys():

            max_id = max(list(self.userid_to_id.values()))
            max_id += 1
            
            self.userid_to_id.update({user_id: max_id})
            self.id_to_userid.update({max_id: user_id})

    def _get_similar_item(self, item_id):
        """Находит товар, похожий на item_id"""
        recs = self.model.similar_items(self.itemid_to_id[item_id], N=2)  # Товар похож на себя -> рекомендуем 2 товара
        top_rec = recs[0][1]  # И берем второй (не товар из аргумента метода)
        return self.id_to_itemid[top_rec]
    
    def _get_similar_user(self, user_id, N):
        """Находит юзеров, похожих на user_id"""
        recs = self.model.similar_users(self.userid_to_id[user_id], N+1)  # Юзер похож на себя -> рекомендуем N+1 юзера
        top_rec = recs[0][1:]  # И берем со второго (не юзер из аргумента метода)
#         for i in range(len(top_rec)):
#             top_rec[i] = self.id_to_userid[top_rec]
        return [(lambda x:self.id_to_userid[x])(x) for x in top_rec]
#             self.id_to_userid[top_rec[:]]

    def _extend_with_top_popular(self, recommendations, N=5):
        """Если кол-во рекоммендаций < N, то дополняем их топ-популярными"""

        if len(recommendations) < N:
            top_popular = [rec for rec in self.overall_top_purchases[:N] if rec not in recommendations and rec!=self.fake_id]
            recommendations.extend(top_popular)
            recommendations = recommendations[:N]

        return recommendations

    def _get_recommendations(self, user, model, N=5):
        """Рекомендации через стардартные библиотеки implicit"""
        self._update_dict(user_id=user)
#         filter_items = [] if self.fake_id is None else [self.itemid_to_id[self.fake_id]]
        res = model.recommend(userid=self.userid_to_id[user],
            user_items=self.user_item_matrix_for_pred[self.userid_to_id[user]],
            N=N,
            filter_already_liked_items=False,
            filter_items=[self.itemid_to_id[999999]],
            recalculate_user=True)
        mask = res[1].argsort()[::-1]
        res = [self.id_to_itemid[rec] for rec in res[0][mask]]
        res = self._extend_with_top_popular(res, N=N)
            
        
        assert len(res) == N, 'Количество рекомендаций != {}'.format(N)
        return res

    def get_als_recommendations(self, user, N=5):
        """Рекомендации через стардартные библиотеки implicit"""
        if user not in self.userid_to_id:
            return self._extend_with_top_popular([], N=N)
        return self._get_recommendations(user, model=self.model, N=N)

    def get_own_recommendations(self, user, N=5):
        """Рекомендуем товары среди тех, которые юзер уже купил"""
        if user not in self.userid_to_id:
            return self._extend_with_top_popular([], N=N)
        return self._get_recommendations(user, model=self.own_recommender, N=N)

    def get_similar_items_recommendation(self, user, N=5):
        """Рекомендуем товары, похожие на топ-N купленных юзером товаров"""

        #your code
        res = []
        top_n = self.top_purchases[self.top_purchases['user_id']==user][:N]['item_id'].tolist()
        if top_n is not None:
            for item in top_n:
                similar_item = self._get_similar_item(item)
                if similar_item != self.fake_id:                    
                    res.append(similar_item) 
        if len(res) < N:
            """Если не получилось найти нужное количество похожих товаров, то добавим в список самые популярные товары"""
            res = self._extend_with_top_popular(res, N=N)       

#         assert len(res) == N, 'Количество рекомендаций != {}'.format(N)
        return res

    def get_similar_users_recommendation(self, user, N=5):
        """Рекомендуем топ-N товаров, среди купленных похожими юзерами"""

        #your code
        res = []
#         similar_users_list = []
        if user in self.userid_to_id:
            similar_users_list = self._get_similar_user(user, N) #.tolist()
        for u in similar_users_list:
            top_user = self.top_purchases[self.top_purchases['user_id']==u]['item_id'].tolist()
            if len(top_user) != 0:
                res.append(top_user[0])
        if len(res) < N:
            """Если не получилось найти нужное количество товаров, то добавим в список самые популярные товары"""
            res = self._extend_with_top_popular(res, N=N)
        return res

In [187]:
def prefilter_items(data, take_n_popular=5000, item_features=None):
    # Уберем самые популярные товары (их и так купят)
    popularity = data.groupby('item_id')['user_id'].nunique().reset_index() / data['user_id'].nunique()
    popularity.rename(columns={'user_id': 'share_unique_users'}, inplace=True)

    top_popular = popularity[popularity['share_unique_users'] > 0.2].item_id.tolist()
    data = data[~data['item_id'].isin(top_popular)]

    # Уберем самые НЕ популярные товары (их и так НЕ купят)
    top_notpopular = popularity[popularity['share_unique_users'] < 0.02].item_id.tolist()
    data = data[~data['item_id'].isin(top_notpopular)]

    # Уберем не интересные для рекоммендаций категории (department)
    if item_features is not None:
        department_size = pd.DataFrame(item_features.\
                                        groupby('department')['item_id'].nunique().\
                                        sort_values(ascending=False)).reset_index()

        department_size.columns = ['department', 'n_items']
        rare_departments = department_size[department_size['n_items'] < 150].department.tolist()
        items_in_rare_departments = item_features[item_features['department'].isin(rare_departments)].item_id.unique().tolist()

        data = data[~data['item_id'].isin(items_in_rare_departments)]


    # Уберем слишком дешевые товары (на них не заработаем). 1 покупка из рассылок стоит 60 руб.
    data['price'] = data['sales_value'] / (np.maximum(data['quantity'], 1))
    data = data[data['price'] > 2]

    # Уберем слишком дорогие товарыs
    data = data[data['price'] < 50]

    # Возьмем топ по популярности
    popularity = data.groupby('item_id')['quantity'].sum().reset_index()
    popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

    top = popularity.sort_values('n_sold', ascending=False).head(take_n_popular).item_id.tolist()
    
    # Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
    data.loc[~data['item_id'].isin(top), 'item_id'] = 999999
    
    # ...

    return data
    

def postfilter(recommendations, item_info, N=5):
    """Пост-фильтрация товаров
    
    Input
    -----
    recommendations: list
        Ранжированный список item_id для рекомендаций
    item_info: pd.DataFrame
        Датафрейм с информацией о товарах
    """
    
    # Уникальность
#     recommendations = list(set(recommendations)) - неверно! так теряется порядок
    unique_recommendations = []
    [unique_recommendations.append(item) for item in recommendations if item not in unique_recommendations]
    
    # Разные категории
    categories_used = []
    final_recommendations = []
    
    CATEGORY_NAME = 'sub_commodity_desc'
    for item in unique_recommendations:
        category = item_features.loc[item_features['item_id'] == item, CATEGORY_NAME].values[0]
        
        if category not in categories_used:
            final_recommendations.append(item)
            
        unique_recommendations.remove(item)
        categories_used.append(category)
    
    n_rec = len(final_recommendations)
    if n_rec < N:
        final_recommendations.extend(unique_recommendations[:N - n_rec])
    else:
        final_recommendations = final_recommendations[:N]
    
    assert len(final_recommendations) == N, 'Количество рекомендаций != {}'.format(N)
    return final_recommendations

In [188]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    
    return precision

In [189]:
def ap_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(recommended_list, bought_list)
    
    if sum(flags) == 0:
        return 0
    
    sum_ = 0
    for i in range(1, k+1):
        
        if flags[i - 1] == True:
            p_k = precision_at_k(recommended_list, bought_list, k=i)
            
            sum_ += p_k
            
    result = sum_ / min(len(recommended_list), k)
    
    return result

In [190]:
def map_k(recommended_list_list, bought_list_list, k=5):
    
    ap = list()
    for i in range(len(recommended_list_list)):
        ap.append(ap_k(recommended_list_list[i], bought_list_list[i], k=k))
    
    map_metric = np.mean(ap)
    
    return map_metric

In [191]:
def ma_k_var2(predicted, actual, k=5):
    return np.mean([ap_k(a,p,k) for a,p in zip(actual, predicted)])

In [192]:
users, items, interactions = data.user_id.nunique(), data.item_id.nunique(), data.shape[0]

print('# users: ', users) #пользователи
print('# items: ', items) #товары
print('# interactions: ', interactions) #взаимодействия

# users:  2499
# items:  89051
# interactions:  2396804


In [193]:
# Используем train-test split по времени, а не случайно. Возьмем последние 3 недели в качестве теста
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [194]:
data_train.shape[0], data_test.shape[0]

(2278490, 118314)

In [195]:
result = data_train.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[825123, 831447, 840361, 845307, 852014, 85498..."
1,2,"[854852, 930118, 1077555, 1098066, 5567388, 55..."


In [196]:
test_users = result.shape[0]
new_test_users = len(set(data_test['user_id']) - set(data_train['user_id']))

print('В тестовом дата сете {} юзеров'.format(test_users))
print('В тестовом дата сете {} новых юзеров'.format(new_test_users))

В тестовом дата сете 2499 юзеров
В тестовом дата сете 0 новых юзеров


In [197]:
popularity = data.groupby('item_id')['user_id'].nunique().reset_index() / data['user_id'].nunique()
popularity.rename(columns={'user_id': 'share_unique_users'}, inplace=True)

top_popular = popularity[popularity['share_unique_users'] > 0.3].item_id.tolist()

In [198]:
def popularity_recommendation(data, n=5):
    """Топ-n популярных товаров"""
    
    popular = data.groupby('item_id')['sales_value'].sum().reset_index()
    popular.sort_values('sales_value', ascending=False, inplace=True)
    
    recs = popular.head(n).item_id
    
    return recs.tolist()

In [199]:
%%time

# Можно так делать, так как рекомендация не зависит от юзера
popular_recs = popularity_recommendation(data_train, n=5)

result['popular_recommendation'] = result['user_id'].apply(lambda x: popular_recs)
result.head(2)

CPU times: user 171 ms, sys: 0 ns, total: 171 ms
Wall time: 172 ms


,user_id,actual,popular_recommendation
0,1,"[825123, 831447, 840361, 845307, 852014, 85498...","[6534178, 6533889, 1029743, 6534166, 1082185]"
1,2,"[854852, 930118, 1077555, 1098066, 5567388, 55...","[6534178, 6533889, 1029743, 6534166, 1082185]"


In [200]:
n_items_before = data_train_lvl_1['item_id'].nunique()

data_train_lvl_1 = prefilter_items(data_train_lvl_1, item_features=item_features, take_n_popular=5000)

n_items_after = data_train_lvl_1['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 83685 to 5001


In [201]:
recommender = MainRecommender(data_train_lvl_1)

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

In [202]:
recommender.get_als_recommendations(2375, N=5)

[1106523, 1029743, 1044078, 899624, 844179]

In [203]:
recommender.get_own_recommendations(2375, N=5)

[1029743, 1106523, 5569230, 916122, 844179]

In [204]:
recommender.get_similar_items_recommendation(2375, N=5)

[1046545, 5591777, 999270, 843111, 1075470]

In [205]:
recommender.get_similar_users_recommendation(2375, N=5)

[899624, 899624, 5569230, 1000753, 1106523]

In [206]:
# В тестовом датасете есть "холодные" покупатели на которах модель не обучалась
# Найдем данные по таким покупателям, для них по умолчанию будем рекомендовать самые популярные товары
list_out = result.loc[~result.user_id.isin(data_train_lvl_1.user_id), 'user_id'].tolist()
list_out

[296, 1813, 1984, 2259]

In [207]:
def rule(x, y, model):
    if model == 'als':
        if x in y:
            return list([840361, 1029743, 995242, 981760, 1082185])
        else:
            return recommender.get_als_recommendations(x, N=5)
    elif model == 'own':
        if x in y:
            return list([840361, 1029743, 995242, 981760, 1082185])
        else:
            return recommender.get_own_recommendations(x, N=5)
    elif model == 'similar_items':
        if x in y:
            return list([840361, 1029743, 995242, 981760, 1082185])
        else:
            return recommender.get_similar_items_recommendation(x, N=5)
    elif model == 'similar_users':
        if x in y:
            return list([840361, 1029743, 995242, 981760, 1082185])
        else:
            return recommender.get_similar_users_recommendation(x, N=5)

In [208]:
result['als'] = result['user_id'].apply(lambda x: rule(x, list_out, model='als'))

In [209]:
result['own'] = result['user_id'].apply(lambda x: rule(x, list_out, model='own'))

In [210]:
result['similar_items'] = result['user_id'].apply(lambda x: rule(x, list_out, model='similar_items'))

In [211]:
result.apply(lambda row: precision_at_k(row['als'], row['actual']), axis=1).mean() * 100

58.719487795118056

In [212]:
result.apply(lambda row: precision_at_k(row['own'], row['actual']), axis=1).mean() * 100

42.90516206482593

In [213]:
result.apply(lambda row: precision_at_k(row['similar_items'], row['actual']), axis=1).mean() * 100

29.891956782713088

In [214]:
result.head(2)

,user_id,actual,popular_recommendation,als,own,similar_items
0,1,"[825123, 831447, 840361, 845307, 852014, 85498...","[6534178, 6533889, 1029743, 6534166, 1082185]","[885290, 5569374, 856942, 940947, 1029743]","[1029743, 1106523, 5569230, 916122, 844179]","[959455, 5582712, 9297615, 1089225, 9803483]"
1,2,"[854852, 930118, 1077555, 1098066, 5567388, 55...","[6534178, 6533889, 1029743, 6534166, 1082185]","[5569230, 1106523, 1029743, 8090521, 8090537]","[1029743, 1106523, 5569230, 916122, 844179]","[8090537, 1021324, 5591777, 897954, 880888]"


Обучаем модель 2-ого уровня на выбранных кандидатах

In [215]:
users_lvl_2 = pd.DataFrame(data_train_lvl_2['user_id'].unique())
users_lvl_2.columns = ['user_id']

# Пока только warm start
train_users = data_train_lvl_1['user_id'].unique()
users_lvl_2 = users_lvl_2[users_lvl_2['user_id'].isin(train_users)]

users_lvl_2['candidates'] = users_lvl_2['user_id'].apply(lambda x: recommender.get_own_recommendations(x, N=50))
users_lvl_2.head(2)

,user_id,candidates
0,2070,"[1029743, 1106523, 5569230, 916122, 844179, 10..."
1,2021,"[1029743, 1106523, 5569230, 916122, 844179, 10..."


In [216]:
users_lvl_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2151 entries, 0 to 2153
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     2151 non-null   int64 
 1   candidates  2151 non-null   object
dtypes: int64(1), object(1)
memory usage: 50.4+ KB


In [217]:
df=pd.DataFrame({'user_id':users_lvl_2.user_id.values.repeat(len(users_lvl_2.candidates[0])),
                 'item_id':np.concatenate(users_lvl_2.candidates.values)})

In [218]:
targets_lvl_2 = data_train_lvl_2[['user_id', 'item_id']].copy()
targets_lvl_2['target'] = 1  # тут только покупки 

targets_lvl_2 = df.merge(targets_lvl_2, on=['user_id', 'item_id'], how='left')

targets_lvl_2['target'].fillna(0, inplace= True)

In [219]:
targets_lvl_2.shape

(110198, 3)

In [220]:
targets_lvl_2 = targets_lvl_2.merge(item_features, on='item_id', how='left') #
targets_lvl_2 = targets_lvl_2.merge(user_features, on='user_id', how='left') #

targets_lvl_2.head(2)

,user_id,item_id,target,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc
0,2070,1029743,0.0,69,GROCERY,Private,FLUID MILK PRODUCTS,FLUID MILK WHITE ONLY,1 GA,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown
1,2070,1106523,0.0,69,GROCERY,Private,FLUID MILK PRODUCTS,FLUID MILK WHITE ONLY,1 GA,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown


In [221]:
result_lvl_2 = data_val_lvl_2.groupby('user_id')['item_id'].unique().reset_index()
result_lvl_2.columns=['user_id', 'actual']

In [222]:
targets_lvl_2.shape

(110198, 16)

In [223]:
targets_lvl_2.fillna(0, inplace=True)

In [224]:
X_train = targets_lvl_2.drop('target', axis=1)
y_train = targets_lvl_2[['target']]

In [225]:
cat_feats = X_train.columns[2:].tolist()
X_train[cat_feats] = X_train[cat_feats].astype('category')

cat_feats

['manufacturer',
 'department',
 'brand',
 'commodity_desc',
 'sub_commodity_desc',
 'curr_size_of_product',
 'age_desc',
 'marital_status_code',
 'income_desc',
 'homeowner_desc',
 'hh_comp_desc',
 'household_size_desc',
 'kid_category_desc']

In [226]:
lgb = LGBMClassifier(objective='binary', 
                     n_estimators=1500,
                     learning_rate=0.008,
                     max_depth=7, 
                     verbose=1,
                     categorical_column=cat_feats)

hist = lgb.fit(X_train, y_train, 
               eval_set=[(X_train, y_train)],
               eval_metric=['logloss'],
               early_stopping_rounds=50,
               verbose=0)

train_preds = lgb.predict_proba(X_train)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


In [227]:
hist.evals_result_['training']['binary_logloss'][-10:]

[0.19760595903313916,
 0.19759657286488697,
 0.1975810779826656,
 0.19756334473562084,
 0.1975479799271027,
 0.1975305870503525,
 0.197513757709738,
 0.1975056252516729,
 0.19749852750754873,
 0.1974847535893347]

In [228]:
classifer_prediction = X_train[['user_id', 'item_id']]
classifer_prediction["pred"] = train_preds[:, 1]
classifer_prediction.tail()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,user_id,item_id,pred
110193,1745,1122358,0.095605
110194,1745,1132771,0.108375
110195,1745,823704,0.020930
110196,1745,854405,0.046041
110197,1745,909714,0.089961


In [229]:
res_val = classifer_prediction.loc[classifer_prediction["pred"]>0.5, ["user_id", "item_id"]]
res_val.shape

(874, 2)

In [230]:
pred_lvl_2 = res_val.groupby('user_id')['item_id'].unique().reset_index()
pred_lvl_2.columns=['user_id', 'pred']
pred_lvl_2.head()

,user_id,pred
0,8,[1029743]
1,13,[1029743]
2,18,[1029743]
3,19,[1029743]
4,20,"[1029743, 1106523, 1126899, 1070820, 1075368]"


In [231]:
result_lvl_2 = result_lvl_2.merge(pred_lvl_2, on='user_id', how='left')
result_lvl_2.head()

,user_id,actual,pred
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...",NaN
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...",NaN
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...",NaN
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...",NaN
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...",[1029743]


In [232]:
user_pred_nan = result_lvl_2.loc[result_lvl_2["pred"] != result_lvl_2["pred"], 'user_id'].values
len(user_pred_nan)

1748

In [233]:
set_tr_1 = set(data_train_lvl_1.user_id.unique())
len(set_tr_1)

2495

In [234]:
user_list = set(user_pred_nan).intersection(set_tr_1)
len(user_list)

1746

In [235]:
result_lvl_2.loc[result_lvl_2["user_id"].isin(user_list), "pred"] =\
result_lvl_2.loc[result_lvl_2["user_id"].isin(user_list), "user_id"].apply(lambda x: recommender.get_als_recommendations(x, N=50))
result_lvl_2.head()

,user_id,actual,pred
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[885290, 5569374, 856942, 940947, 1029743, 102..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1106523, 1029743, 1044078, 8090521, 5569230, ..."
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[878996, 965267, 1026118, 866211, 854852, 9869..."
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[1029743, 916122, 1106523, 1126899, 1122358, 8..."
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...",[1029743]


In [236]:
result_lvl_2.loc[result_lvl_2["user_id"] == 792]

,user_id,actual,pred
633,792,"[1144192, 1303894, 1314715, 1331582, 1343866, ...","[1029743, 1106523, 5569230, 1044078, 5569471, ..."


In [237]:
popularity = recommender.top_purchases.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)
top_5 = popularity.sort_values('n_sold', ascending=False).head(6).item_id.tolist()[1:6]
top_5

[1106523, 5569230, 916122, 844179, 1044078]

In [238]:
result_lvl_2.loc[result_lvl_2["user_id"] == 792, "pred"] =\
        result_lvl_2.loc[result_lvl_2["user_id"] == 792, "user_id"].apply(lambda x: top_5)

In [239]:
result_lvl_2.loc[result_lvl_2["user_id"] == 792]

,user_id,actual,pred
633,792,"[1144192, 1303894, 1314715, 1331582, 1343866, ...","[1106523, 5569230, 916122, 844179, 1044078]"


In [240]:
len(result_lvl_2.loc[result_lvl_2["user_id"] == 792, "pred"].values[0])

5

In [241]:
res = []
res.extend(result_lvl_2.loc[result_lvl_2["user_id"] == 792, "pred"].values[0])
res

[1106523, 5569230, 916122, 844179, 1044078]

In [242]:
result.head(2)

,user_id,actual,popular_recommendation,als,own,similar_items
0,1,"[825123, 831447, 840361, 845307, 852014, 85498...","[6534178, 6533889, 1029743, 6534166, 1082185]","[885290, 5569374, 856942, 940947, 1029743]","[1029743, 1106523, 5569230, 916122, 844179]","[959455, 5582712, 9297615, 1089225, 9803483]"
1,2,"[854852, 930118, 1077555, 1098066, 5567388, 55...","[6534178, 6533889, 1029743, 6534166, 1082185]","[5569230, 1106523, 1029743, 8090521, 8090537]","[1029743, 1106523, 5569230, 916122, 844179]","[8090537, 1021324, 5591777, 897954, 880888]"


In [243]:
result.apply(lambda row: precision_at_k(row['popular_recommendation'], row['actual']), axis=1).mean() * 100

39.303721488595436

In [244]:
result.apply(lambda row: precision_at_k(row['own'], row['actual']), axis=1).mean() * 100

42.90516206482593

In [245]:
result.apply(lambda row: precision_at_k(row['als'], row['actual']), axis=1).mean() * 100

58.719487795118056

In [246]:
result.apply(lambda row: precision_at_k(row['similar_items'], row['actual']), axis=1).mean() * 100

29.891956782713088

In [247]:
result.apply(lambda row: ap_k(row['popular_recommendation'], row['actual']), axis=1).mean() * 100

24.952380952380953

In [248]:
result.apply(lambda row: ap_k(row['own'], row['actual']), axis=1).mean() * 100

32.80325463518741

In [249]:
result.apply(lambda row: ap_k(row['als'], row['actual']), axis=1).mean() * 100

49.439375750300115

In [250]:
result.apply(lambda row: ap_k(row['similar_items'], row['actual']), axis=1).mean() * 100

19.486461251167135

Прогнозирование на тестовом датасете

In [252]:
result_test = test_df.groupby('user_id')['item_id'].unique().reset_index()
result_test.columns=['user_id', 'actual']
result_test.head(2)

,user_id,actual
0,1,"[880007, 883616, 931136, 938004, 940947, 94726..."
1,2,"[820165, 820291, 826784, 826835, 829009, 85784..."


In [254]:
common_users = data_train.user_id.values

In [255]:
result_test = result_test[result_test.user_id.isin(common_users)]

In [256]:
%%time
result_test['own'] = result_test['user_id'].apply(lambda x: rule(x, list_out, model='own'))

CPU times: user 312 ms, sys: 0 ns, total: 312 ms
Wall time: 317 ms


In [257]:
result_test.apply(lambda row: precision_at_k(row['own'], row['actual']), axis=1).mean() * 100

8.386411889596603

In [258]:
result_test.apply(lambda row: ap_k(row['own'], row['actual']), axis=1).mean() * 100

4.993630573248408